<a href="https://colab.research.google.com/github/GarimaChaubey/Fake-News-Classifier-Using-LSTM/blob/main/FakeNewsClassifierUsingBirdirectionalLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd

In [34]:
df=pd.read_csv('train.csv')

In [35]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [36]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [37]:
df.shape

(20800, 5)

In [38]:
df=df.dropna()

In [39]:
df.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [40]:
X=df.drop('label',axis=1)

In [41]:
y=df['label']

In [42]:
y.value_counts()

,count
label,
0,10361
1,7924


In [43]:
import tensorflow as tf

In [44]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [45]:
voc_size=5000

In [46]:
messages=X.copy()

In [47]:
import nltk
import re
from nltk.corpus import stopwords

In [48]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
messages.reset_index(inplace=True)

In [50]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
  review=re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review=review.lower()
  review=review.split()

  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [51]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [52]:
onehot_repr=[one_hot(words, voc_size) for words in corpus]
onehot_repr

[[2890, 277, 4622, 2937, 3612, 4343, 2181, 366, 3706, 1105],
 [1539, 83, 4886, 2477, 2143, 1736, 1450],
 [4359, 2048, 2432, 655],
 [2822, 2369, 91, 4078, 2730, 2890],
 [3963, 2143, 3778, 3987, 4214, 715, 2143, 3723, 1712, 4601],
 [3773,
  2614,
  901,
  4653,
  3409,
  398,
  2217,
  2327,
  4520,
  1403,
  948,
  962,
  2716,
  3301,
  1450],
 [1836, 2430, 4326, 4816, 183, 2268, 4961, 1055, 3250, 3472, 401],
 [2168, 4162, 2817, 28, 2823, 2726, 398, 2614, 3250, 3472, 401],
 [1674, 3440, 1456, 2840, 665, 3997, 3453, 2103, 398, 152],
 [3935, 815, 800, 2906, 4326, 3641, 4340, 2415],
 [3606, 2509, 4408, 1760, 1905, 2953, 3070, 3104, 3000, 4412, 3027],
 [4078, 1818, 3612, 3997, 398, 2823],
 [4651, 3239, 3019, 4598, 331, 1151, 4972, 1270, 4916],
 [2452, 4770, 2173, 1669, 4048, 2179, 2466, 3250, 3472, 401],
 [3573, 4495, 2313, 3285, 4178, 3250, 3472, 401],
 [1726, 1379, 2343, 1300, 2602, 4649, 3133, 1030, 2468, 2231],
 [2237, 868, 83],
 [2812, 204, 4158, 4221, 398, 1121, 2154, 1450],
 [2299, 

# Embedding Representation

In [53]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)



In [54]:
embedded_docs

array([[   0,    0,    0, ...,  366, 3706, 1105],
       [   0,    0,    0, ..., 2143, 1736, 1450],
       [   0,    0,    0, ..., 2048, 2432,  655],
       ...,
       [   0,    0,    0, ..., 3250, 3472,  401],
       [   0,    0,    0, ..., 4127,  696, 4468],
       [   0,    0,    0, ...,  819, 2610, 2256]], dtype=int32)

In [55]:
 #Creating model

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [56]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [58]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.8117 - loss: 0.3728 - val_accuracy: 0.9183 - val_loss: 0.1870
Epoch 2/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9490 - loss: 0.1307 - val_accuracy: 0.9223 - val_loss: 0.1929
Epoch 3/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9731 - loss: 0.0778 - val_accuracy: 0.9216 - val_loss: 0.2331
Epoch 4/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9843 - loss: 0.0495 - val_accuracy: 0.9176 - val_loss: 0.2523
Epoch 5/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9940 - loss: 0.0229 - val_accuracy: 0.9155 - val_loss: 0.3330
Epoch 6/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.9960 - loss: 0.0154 - val_accuracy: 0.9162 - val_loss: 0.3673
Epoch 7/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9967 - loss: 0.0110 - val_accuracy: 0.9102 - val_loss: 0.4371
Epoch 8/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9982 - loss: 0.0075 - val_accuracy

In [59]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [60]:
import numpy as np

y_pred=np.where=(y_pred>0.5, 1,0)

In [62]:
from sklearn.metrics import accuracy_score